#### The question is: 
### Are these equivalent?
1. **Convolutions** -(then)-> **Window split**
2. **Window split** -(then)-> **Convolutions**

In [1]:
import torch
from torch_geometric.data import Data

node_features = torch.tensor([[2,3],[-1,0],[4,7],[5,9]]).float()
source_nodes = torch.tensor([0,0,1,1,2,2,2,3])
target_nodes = torch.tensor([1,2,0,2,0,1,3,2])
edge_index = torch.stack([source_nodes,target_nodes])
y = torch.tensor([1])

# Graph object
g = Data(x=node_features,edge_index=edge_index, y=y)
g

Data(edge_index=[2, 8], x=[4, 2], y=[1])

In [2]:
from torch_geometric.nn import GraphConv

conv = GraphConv(in_channels=2, out_channels=2, aggr='add', bias=False)
conv(g.x.float(),g.edge_index)

tensor([[ 5.6625,  0.6603],
        [ 7.4924, -1.2082],
        [10.9991,  1.8233],
        [ 8.7213,  3.1475]], grad_fn=<AddBackward0>)

In [3]:
params = []
for p in (conv.parameters()):
    params.append(p)
params

[Parameter containing:
 tensor([[ 0.3443,  0.4988],
         [-0.0094, -0.0946]], requires_grad=True),
 Parameter containing:
 tensor([[-0.4386,  0.6717],
         [ 0.2053,  0.3135]], requires_grad=True)]

In [4]:
torch.matmul(params[0],node_features[0])

tensor([ 2.1850, -0.3028], grad_fn=<MvBackward>)

In [5]:
conv(g.x.float(),g.edge_index)

tensor([[ 5.6625,  0.6603],
        [ 7.4924, -1.2082],
        [10.9991,  1.8233],
        [ 8.7213,  3.1475]], grad_fn=<AddBackward0>)

In [6]:
# Feature aggregation for neighbors to node 0
neighbor_aggr = torch.sum(torch.stack([node_features[1],node_features[2]]),dim=0)
neighbor_aggr

tensor([3., 7.])

In [8]:
torch.matmul(params[1],node_features[0]) + torch.matmul(params[0],neighbor_aggr)

tensor([5.6625, 0.6603], grad_fn=<AddBackward0>)

## They are equivalent